In [62]:
import dotenv
from langchain_openai import ChatOpenAI
dotenv.load_dotenv()
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders.csv_loader import CSVLoader


chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [63]:

messages = [
    SystemMessage(
        content="""You're an assistant knowledgeable about
        stockmarket. Only answer stock market."""
    ),
    HumanMessage(content="What are top 10 stocks which can give good returns?"),
]
#response = chat_model(messages)
#print(response.content)
chat_model.invoke(messages)

AIMessage(content="I'm sorry, but I cannot provide specific stock recommendations. It's important to do thorough research and consider various factors before investing in any stock. I recommend consulting with a financial advisor or doing your own research to determine which stocks may be suitable for your investment goals and risk tolerance.", response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 39, 'total_tokens': 95}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-60730080-8aa3-4c73-a1a2-29bcded2084c-0')

In [71]:
review_template_str = """Your job is to use news about stocks and their sentiment to recommend stocks.
Use the following context to answer questions. Be as detailed
as possible, but don't make up any information that's not
from the context. do not give anything that is not in context.If you don't know an answer, say you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"],
        template=review_template_str,
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],
        template="{question}",
    )
)
messages = [review_system_prompt, review_human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)

chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
output_parser = StrOutputParser()
review_chain = review_prompt_template | chat_model | output_parser

In [65]:
from langchain.chat_models import ChatOpenAI

context = "information on stocks"
question = "list of best stocks in Indian stock market"

review_chain.invoke({"context": context, "question": question})

"As an AI language model, I don't have real-time data on the stock market. However, I can provide you with some popular and well-known stocks in the Indian stock market that have historically performed well. Please note that the performance of these stocks can vary, and it's essential to do your research or consult with a financial advisor before making any investment decisions. Here are some of the well-known stocks in the Indian stock market:\n\n1. Reliance Industries Limited (RIL): RIL is one of the largest conglomerates in India with businesses across various sectors like petrochemicals, refining, retail, and telecommunications.\n\n2. Tata Consultancy Services (TCS): TCS is a leading IT services company and a part of the Tata Group. It is one of the largest IT services firms globally.\n\n3. HDFC Bank: HDFC Bank is one of the largest private sector banks in India known for its strong financial performance and customer service.\n\n4. Infosys: Infosys is another prominent IT services 

In [73]:

REVIEWS_CSV_PATH = "E:/Data Science/Capstone/news.csv"
REVIEWS_CHROMA_PATH = "E:/Data Science/Capstone/chroma_data"


loader = CSVLoader(file_path=REVIEWS_CSV_PATH, source_column="text")
reviews = loader.load()

reviews_vector_db = Chroma.from_documents(
    reviews, OpenAIEmbeddings(), persist_directory=REVIEWS_CHROMA_PATH
)

In [67]:


reviews_vector_db = Chroma(
    persist_directory=REVIEWS_CHROMA_PATH,
    embedding_function=OpenAIEmbeddings(),
)
question = """give me a stock which has high sentiment"""
relevant_docs = reviews_vector_db.similarity_search(question, k=3)

relevant_docs[0].page_content
relevant_docs[1].page_content
relevant_docs[2].page_content

'text: The Indian stock market has shown robust growth, rising 1.5% in the past week and an impressive 46% over the last 12 months, with earnings expected to grow by 16% annually. In this flourishing environment, stocks with high insider ownership can be particularly appealing as they often indicate a strong alignment between company management and shareholder interests.'

In [74]:
question = """give me stock with huge upside potentential and low risk from indian stock market"""

# Assuming the prompt expects a key 'question'
review_chain.invoke({"context": context, "question": question})


'One stock that has been identified as having a huge upside potential with relatively low risk in the Indian stock market is Infosys Limited. Infosys is a well-established multinational corporation that provides information technology, consulting, and outsourcing services. The company has a strong track record of consistent growth and profitability, making it a relatively low-risk investment option.\n\nInfosys has been performing well in recent years, with a strong focus on digital transformation services and a solid client base. The company has also been investing in emerging technologies such as artificial intelligence, cloud computing, and cybersecurity, which are expected to drive future growth.\n\nAnalysts have a positive outlook on Infosys, with many recommending it as a buy due to its strong fundamentals and growth potential. The stock is considered to have a significant upside potential, making it an attractive investment option for investors looking for growth opportunities wi